In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.font_manager as fm
import urllib.request
import os

In [ ]:
file_path = 'keyword_final.csv'
data = pd.read_csv(file_path)

# 날짜 열을 datetime 형식으로 변환
data['일자'] = pd.to_datetime(data['일자'])

# 연도와 분기 추출
data['분기'] = data['일자'].dt.quarter

# 연도와 분기별로 그룹화하고 키워드를 리스트로 결합
quarterly_keywords = data.groupby(['분기'])['키워드'].apply(lambda x: ' '.join(x)).reset_index()

# 분기별 키워드 빈도를 계산하는 함수
def get_quarterly_keywords(df):
    vectorizer = CountVectorizer()
    df['키워드_빈도'] = df['키워드'].apply(lambda x: vectorizer.fit_transform([x]).toarray().sum(axis=0))
    df['키워드'] = df['키워드'].apply(lambda x: vectorizer.get_feature_names_out())
    return df

quarterly_keywords = get_quarterly_keywords(quarterly_keywords)

# 분기별 새롭게 등장한 키워드를 찾는 함수
def find_new_keywords(df):
    new_keywords = []
    for i in range(1, len(df)):
        current_keywords = set(df.iloc[i]['키워드'])
        previous_keywords = set(df.iloc[i-1]['키워드'])
        new_in_quarter = current_keywords - previous_keywords
        for keyword in new_in_quarter:
            new_keywords.append({
                '분기': df.iloc[i]['분기'],
                '키워드': keyword
            })
    return pd.DataFrame(new_keywords)

new_keywords = find_new_keywords(quarterly_keywords)

# 분기별 새롭게 등장한 키워드 시각화
def plot_new_keywords(df):
    plt.figure(figsize=(12, 8))
    quarters = df.groupby(['연도', '분기']).size().reset_index().rename(columns={0: 'count'})
    for _, row in quarters.iterrows():
        quarter_keywords = df[(df['연도'] == row['연도']) & (df['분기'] == row['분기'])]
        plt.barh(quarter_keywords['키워드'], range(len(quarter_keywords)), label=f"{row['연도']} Q{row['분기']}")
    
    plt.xlabel('등장 순위')
    plt.ylabel('키워드', fontproperties=fontprop)
    plt.title('분기별 새롭게 등장한 키워드', fontproperties=fontprop)
    plt.legend()
    plt.show()

plot_new_keywords(new_keywords)

# 분기별 키워드를 CSV 파일로 저장
def save_quarterly_keywords(df, output_dir='quarterly_keywords'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    quarters = df.groupby(['분기'])
    for (year, quarter), group in quarters:
        filename = f'{output_dir}/{year}_Q{quarter}_keywords.csv'
        group.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"Saved {filename}")

save_quarterly_keywords(new_keywords)